In [12]:
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler,RobustScaler
from geopy.distance import geodesic
import geopy.geocoders
from IPython.core.display import display, HTML

gl = geopy.geocoders.Nominatim(user_agent="geoapi")

region={
      "DDA Market": "Delhi",
    "Ghazipur Market": "Delhi",
    "Barnala Market": "Punjab",
    "Ludhiana Market": "Punjab",
    "Sunam Dhuri Market": "Punjab",
    "Amritsar Market": "Punjab",
    "Faridabad Market": "Haryana",
    "Yamuna Nagar Market": "Haryana",
      #  "Donar Wholesale Market":"Bihar",
        "Macchwatoli Retail Fish Market":"Bihar",
        "Mushallapur Wholesale Fish Market":"Bihar",
        "Saidnagar Retail Fish Market":"Bihar",
        "Bhakra Gobind Sagar Fish Landing Center":"Himachal Pradesh",
        "Nagrota Surian Fish Landing Center":"Himachal Pradesh",
        "Fish Retail Market Gumma Shimla":"Himachal Pradesh",
        "Jaipur Wholesale Fish Market":"Rajasthan",
        "Udaipur Retail Fish Market":"Rajasthan",
        "Wholesale Market Dubagga Lucknow":"Uttar Pradesh",
        "Retail Market Mahesara Gorakhpur":"Uttar Pradesh",
        # "JK Fish Sahasthadhara Wholesale Market":"Uttrakhand",
        "Naveen Krishi Mandi Wholesale Fish Market":"Uttrakhand"
}

market_mapping = {
        "DDA Market": "MarketA",
        "Ghazipur Market": "MarketB",
        "Barnala Market": "MarketC",
        "Ludhiana Market": "MarketD",
        "Sunam Dhuri Market": "MarketE",
        "Amritsar Market": "MarketF",
        "Faridabad Market": "MarketG",
        "Yamuna Nagar Market": "MarketH",
        # "Donar Wholesale Market":"marketI",
        "Macchwatoli Retail Fish Market":"MarketJ",
        "Mushallapur Wholesale Fish Market":"MarketK",
        "Saidnagar Retail Fish Market":"MarketL",
        "Bhakra Gobind Sagar Fish Landing Center":"MarketM",
        "Nagrota Surian Fish Landing Center":"MarketN",
        "Fish Retail Market Gumma Shimla":"MarketO",
        "Jaipur Wholesale Fish Market":"MarketP",
        "Udaipur Retail Fish Market":"MarketQ",
        "Wholesale Market Dubagga Lucknow":"MarketR",
        "Retail Market Mahesara Gorakhpur":"MarketS",
        # "JK Fish Sahasthadhara Wholesale Market":"marketT",
        "Naveen Krishi Mandi Wholesale Fish Market":"MarketU"
    }

market_locations = {
    "DDA Market": (28.6293400585683, 77.24496833863526),
    "Ghazipur Market": (28.655760730252887, 77.23633716931764),
    "Barnala Market": (30.37456374754022, 75.55523950795289),
    "Ludhiana Market": (30.917512621972246, 75.90517035454104),
    "Sunam Dhuri Market": (30.120285807169914, 75.80614867727054),
    "Amritsar Market": (31.63208335321038, 74.875477),
    "Faridabad Market": (28.369151205973754, 77.29569019394951),
    "Yamuna Nagar Market": (30.1282, 77.2764),
    #  "Donar Wholesale Market":(26.14587849071568, 85.9070944423244),
        "Macchwatoli Retail Fish Market":(25.641290329040284, 85.05050036560492),
        "Mushallapur Wholesale Fish Market":(25.598701647040865, 85.1759847089092),
        "Saidnagar Retail Fish Market":(26.11823500993768, 85.90029226872616),
        "Bhakra Gobind Sagar Fish Landing Center":(31.39662496966552, 76.3829398101351),
        "Nagrota Surian Fish Landing Center":(32.05356293154015, 76.09088150795289),
        "Fish Retail Market Gumma Shimla":(31.118118686775656, 77.49164898409424),
        "Jaipur Wholesale Fish Market":(26.90997259599042, 75.72901233863526),
        "Udaipur Retail Fish Market":(25.53561157589193, 73.87187846102283),
        "Wholesale Market Dubagga Lucknow":(26.879113259527497, 80.84958974210605),
        "Retail Market Mahesara Gorakhpur":(26.823068787831545, 83.3503921173067),
        # "JK Fish Sahasthadhara Wholesale Market":"marketT",
        "Naveen Krishi Mandi Wholesale Fish Market":(26.96827171187265, 80.92989944951847)
}
def get_distance(user_location, market_location):
    return geodesic(user_location, market_location).km

def get_coordinates(location_name):
    try:
        location = gl.geocode(location_name)
        return (location.latitude, location.longitude) if location else None
    except:
        return None

def make_clickable(val):
    return f'<a href="{val}" target="_blank">Route</a>' if "http" in val else val


def compare_markets(fore_date, fish, size, user_location):
    if fore_date < datetime.strptime("07-03-2025", "%d-%m-%Y"):
        return "Enter a date after 7th March 2025 only"

    user_coords = get_coordinates(user_location)
    if not user_coords:
        return "Invalid location. Please enter a valid place."

    market_predictions = []

    train_data = pd.read_excel("north_india_agg_27march.xlsx")
    train_data.dropna(subset=['Date'], inplace=True)
    train_data['Date'] = pd.to_datetime(train_data['Date'], format='%d-%m-%Y', errors='coerce')
    full_dates = pd.DataFrame({'Date': pd.date_range(start=train_data['Date'].min(), end=train_data['Date'].max(), freq='D')})
    train_data = full_dates.merge(train_data, on='Date', how='left')
    train_data.set_index('Date', inplace=True)
    train_data.replace(0, np.nan, inplace=True)
    train_data = train_data.interpolate(method="polynomial",order=2,limit_direction="both")
    train_data = train_data.interpolate(method="linear",limit_direction="both")
    for col in train_data.columns:
      train_data[col]=train_data[col].fillna(train_data[col].median())
    for col in train_data.columns:
      if (train_data[col] < 0).any():
        smooth_mean = train_data.loc[train_data[col] > 0, col].mean()
        train_data[col] = train_data[col].apply(lambda x: x if x > 0 else smooth_mean)
    scaler=RobustScaler()
    train_scaled = pd.DataFrame(scaler.fit_transform(np.cbrt(train_data)),
                                columns=train_data.columns,
                                index=train_data.index)


    with open('model.pkl', 'rb') as file:
        model = pickle.load(file)

    baseline_date = datetime.strptime("07-03-2025", "%d-%m-%Y")
    steps = (fore_date - baseline_date).days
    lag_order = model.k_ar
    forecast_scaled = model.forecast(train_scaled.values[-lag_order:], steps=steps)
    forecast = scaler.inverse_transform(forecast_scaled)
    forecast_data = pd.DataFrame(forecast, columns=train_data.columns)
    forecast_data = np.power(forecast_data, 3)


    for market, coords in market_locations.items():
        fish_and_size = f"{fish}_{size}_{market_mapping[market]}"
        if fish_and_size in forecast_data.columns:
            predicted_price = np.round(forecast_data.loc[0, fish_and_size])
            distance = get_distance(user_coords, coords)
            maps_url = f"https://www.google.com/maps/dir/{user_coords[0]},{user_coords[1]}/{coords[0]},{coords[1]}"
            market_predictions.append({
                "Region": region[market],
                "Market": f"{market}",
                "Prediction in ₹": predicted_price,
                "Distance in km": round(distance, 2),
                "Route": maps_url
            })
        else:
            market_predictions.append({
                "Region": region[market],
                "Market": f"{market}",
                "Prediction in ₹": "Not Sold Here",
                "Distance in km": round(get_distance(user_coords, coords), 2),
                "Route": "N/A"
            })


    market_predictions.sort(key=lambda x: (x["Prediction in ₹"] if isinstance(x["Prediction in ₹"], (int, float)) else -1), reverse=True)


    df = pd.DataFrame(market_predictions)
    df["Route"] = df["Route"].apply(make_clickable)
    display(HTML(df.to_html(escape=False)))




In [13]:
compare_markets(datetime.strptime("30-06-2025", "%d-%m-%Y"),"Catla","Small","Karol Bagh, Delhi")

,Region,Market,Prediction in ₹,Distance in km,Route
0,Bihar,Macchwatoli Retail Fish Market,343.0,847.46,Route
1,Bihar,Saidnagar Retail Fish Market,307.0,906.00,Route
2,Himachal Pradesh,Nagrota Surian Fish Landing Center,280.0,391.48,Route
3,Uttar Pradesh,Retail Market Mahesara Gorakhpur,269.0,640.35,Route
4,Delhi,DDA Market,215.0,6.07,Route
5,Bihar,Mushallapur Wholesale Fish Market,207.0,860.87,Route
6,Delhi,Ghazipur Market,201.0,4.64,Route
7,Punjab,Barnala Market,180.0,247.99,Route
8,Himachal Pradesh,Fish Retail Market Gumma Shimla,180.0,274.82,Route
9,Punjab,Ludhiana Market,150.0,280.03,Route


In [14]:

compare_markets(datetime.strptime("01-05-2025", "%d-%m-%Y"),"Rohu","Medium","gurugram")

,Region,Market,Prediction in ₹,Distance in km,Route
0,Delhi,DDA Market,299.0,27.86,Route
1,Delhi,Ghazipur Market,239.0,29.27,Route
2,Punjab,Sunam Dhuri Market,204.0,218.67,Route
3,Punjab,Barnala Market,200.0,255.53,Route
4,Punjab,Ludhiana Market,199.0,292.87,Route
5,Haryana,Faridabad Market,125.0,28.11,Route
6,Punjab,Amritsar Market,Not Sold Here,407.96,N/A
7,Haryana,Yamuna Nagar Market,Not Sold Here,185.94,N/A
8,Bihar,Macchwatoli Retail Fish Market,Not Sold Here,854.74,N/A
9,Bihar,Mushallapur Wholesale Fish Market,Not Sold Here,868.18,N/A
